# Geo. Display #

### Purposes of this application: ###
- Users Will input a lat lon and a n X m degree heatmap of elevation data will be returned
- The program will work remotelty with the ngrok endpoint through the AWS EC2 instance and be rooted to a pi (hopefully)
- After reciveing the center lat lon, a n X m grid will be made (pre deteremined size) and using the USGS lidar map it will create a 3d heatmap
  

In [2]:
import json
from pyproj import Transformer
import pdal
import numpy as np
import matplotlib.pyplot as plt
import laspy

In [ ]:
# Define the midpoint
# mid_lat, mid_lon = 37.558, -77.527  # Midpoint of william island
# mid_lat, mid_lon = 37.06341617858604, -76.49445533752443  # CNU 
mid_lat, mid_lon = 37.53400769401545, -77.44275569915773  # Browns island
# Define the size of the square in degrees (you can adjust this value)
# For example, here the square will have a side length of 0.01 degrees
square_size = .1  # Size of the square in degrees

# Step 1: Calculate the bounds of the square based on the midpoint
lat_min = mid_lat - (square_size / 2)
lat_max = mid_lat + (square_size / 2)
lon_min = mid_lon - (square_size / 2)
lon_max = mid_lon + (square_size / 2)

# Step 2: Display the bounds of the square
print("Square Bounds based on midpoint:")
print(f"Top Left (Lat, Lon): ({lat_max}, {lon_min})")
print(f"Bottom Right (Lat, Lon): ({lat_min}, {lon_max})")


# Transform the latitude/longitude bounds to EPSG:3857 (Web Mercator)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x_min, y_min = transformer.transform(lon_min, lat_min)
x_max, y_max = transformer.transform(lon_max, lat_max)

# Define the PDAL pipeline JSON with the transformed bounds
pipeline_json = {
    "pipeline": [
        {
            "type": "readers.ept",
            "filename": "http://usgs-lidar-public.s3.amazonaws.com/USGS_LPC_VA_Sandy_2014_LAS_2015/ept.json",
            "bounds": f"([{x_min}, {x_max}], [{y_min}, {y_max}])"
        },
        {
            "type": "writers.las",
            "filename": "/work/bkelley/large_data/geospatial/data/browns_island.laz"
        }
    ]
}

# Convert the pipeline to JSON string format
pipeline = json.dumps(pipeline_json)

# Run the PDAL pipeline
p = pdal.Pipeline(pipeline)
p.execute()

print("LAZ file created: browns_island.laz")


In [ ]:
# Load the .laz file
file_path = "/work/bkelley/large_data/geospatial/data/browns_island.laz"
with laspy.open(file_path) as f:
    point_data = f.read()

# Convert x, y, and z to numpy arrays to avoid compatibility issues
x = np.array(point_data.x)
y = np.array(point_data.y)
z = np.array(point_data.z)  # Or use `intensity` if you want to plot intensity

# Create a 2D histogram (heatmap) of the point density
plt.figure(figsize=(10, 8))
plt.hist2d(x, y, bins=500, weights=z, cmap='viridis')  # Adjust bins and cmap as needed
plt.colorbar(label="Elevation (or Intensity)")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title("2D Heatmap of Point Cloud Elevation Data")
plt.show()
